In [25]:
import torch
print(torch.__version__)

1.4.0


In [26]:
import sys 
sys.path.append('../src/')
import load_transform_data
from torch.utils import data 
from torch import nn
import torch.nn.functional as F
import pandas as pd 

from my_classes import Dataset 

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Parameters 
train_params = {'batch_size': 64, 'shuffle': True, 'num_workers': 0}
test_params = {'batch_size': 1, 'shuffle': True, 'num_workers': 0}

# Generators 
trans = load_transform_data.TransformXY()
trans.transform_data('LabelBinarizer')

train_set = Dataset(trans, 'train')
train_generator = data.DataLoader(train_set, **train_params)

test_set = Dataset(trans, 'test')
test_generator = data.DataLoader(test_set, **test_params)

In [27]:
# CONSTANTS
max_epochs = 300
n_feature = 64
n1_hidden = 48
n2_hidden = 48
n1_out = 8  # train_set.Y_tone.shape 
n2_out = 18 # train_set.Y_initial.shape
n3_out = 70 # train_set.Y_final.shape
p_dropout1 = 0.5
p_dropout2 = 0.5

In [28]:
class TeochewNet(nn.Module):
    def __init__(self, n_feature, n1_hidden, n2_hidden, n1_out, n2_out, n3_out):
        """"Constructor of the class"""
        super(TeochewNet, self).__init__()
        
        self.hidden1 = torch.nn.Linear(n_feature, n1_hidden)
        self.hidden2 = torch.nn.Linear(n1_hidden, n2_hidden)
        self.dropout1 = torch.nn.Dropout(p_dropout1)
        self.dropout2 = torch.nn.Dropout(p_dropout2)
        self.relu = nn.ReLU()
        self.out1 = torch.nn.Linear(n2_hidden, n1_out)
        self.out2 = torch.nn.Linear(n2_hidden, n2_out)
        self.out3 = torch.nn.Linear(n2_hidden, n3_out)
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x1 = self.out1(x)
        x2 = self.out2(x)
        x3 = self.out3(x)
        return x1.type(torch.DoubleTensor), x2.type(torch.DoubleTensor), x3.type(torch.DoubleTensor)
    
teochewnet = TeochewNet(n_feature, n1_hidden, n2_hidden, n1_out, n2_out, n3_out)
print(teochewnet)

TeochewNet(
  (hidden1): Linear(in_features=64, out_features=48, bias=True)
  (hidden2): Linear(in_features=48, out_features=48, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
  (out1): Linear(in_features=48, out_features=8, bias=True)
  (out2): Linear(in_features=48, out_features=18, bias=True)
  (out3): Linear(in_features=48, out_features=70, bias=True)
)


In [29]:
from torch.autograd import Variable
import torch.optim as optim

criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()

optimizer = optim.Adam(teochewnet.parameters(), lr=0.01, weight_decay=0.01)

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

for epoch in range(max_epochs):  # loop over the dataset multiple times
    epoch_loss = 0.0
    epoch_acc = 0
    for i, (x, y_tone, y_initial, y_final) in enumerate(train_generator, 0):
        # zero the parameter gradients
        out1, out2, out3 = teochewnet(Variable(x.float()))
        loss1 = criterion1(out1, Variable(y_tone))
        loss2 = criterion2(out2, Variable(y_initial))
        loss3 = criterion3(out3, Variable(y_final))
        loss = loss1 + loss2 + loss3 
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
        
        # print statistics
        epoch_loss += loss.item()
        tone_acc = categorical_accuracy(out1, y_tone)
        initial_acc = categorical_accuracy(out2, y_initial)
        final_acc = categorical_accuracy(out3, y_final)

        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f tone acc: %.3f initial acc: %.3f final acc: %.3f' %
                  (epoch + 1, i + 1, epoch_loss / 10, tone_acc, initial_acc, final_acc))
            epoch_loss = 0.0
            
print('Finished Training')

[1,    10] loss: 8.995 tone acc: 0.281 initial acc: 0.094 final acc: 0.062
[2,    10] loss: 8.443 tone acc: 0.391 initial acc: 0.188 final acc: 0.078
[3,    10] loss: 8.083 tone acc: 0.484 initial acc: 0.203 final acc: 0.062
[4,    10] loss: 7.830 tone acc: 0.641 initial acc: 0.188 final acc: 0.078
[5,    10] loss: 7.656 tone acc: 0.578 initial acc: 0.219 final acc: 0.109
[6,    10] loss: 7.593 tone acc: 0.578 initial acc: 0.281 final acc: 0.094
[7,    10] loss: 7.426 tone acc: 0.734 initial acc: 0.203 final acc: 0.078
[8,    10] loss: 7.364 tone acc: 0.609 initial acc: 0.266 final acc: 0.062
[9,    10] loss: 7.285 tone acc: 0.562 initial acc: 0.250 final acc: 0.078
[10,    10] loss: 7.168 tone acc: 0.625 initial acc: 0.250 final acc: 0.094
[11,    10] loss: 7.196 tone acc: 0.562 initial acc: 0.406 final acc: 0.047
[12,    10] loss: 7.071 tone acc: 0.641 initial acc: 0.344 final acc: 0.047
[13,    10] loss: 7.039 tone acc: 0.531 initial acc: 0.359 final acc: 0.078
[14,    10] loss: 6.9

[109,    10] loss: 6.125 tone acc: 0.609 initial acc: 0.500 final acc: 0.125
[110,    10] loss: 6.147 tone acc: 0.625 initial acc: 0.531 final acc: 0.219
[111,    10] loss: 6.169 tone acc: 0.625 initial acc: 0.500 final acc: 0.078
[112,    10] loss: 6.105 tone acc: 0.609 initial acc: 0.516 final acc: 0.172
[113,    10] loss: 6.135 tone acc: 0.531 initial acc: 0.406 final acc: 0.109
[114,    10] loss: 6.075 tone acc: 0.609 initial acc: 0.547 final acc: 0.125
[115,    10] loss: 6.131 tone acc: 0.609 initial acc: 0.531 final acc: 0.109
[116,    10] loss: 6.154 tone acc: 0.656 initial acc: 0.391 final acc: 0.188
[117,    10] loss: 6.054 tone acc: 0.625 initial acc: 0.562 final acc: 0.078
[118,    10] loss: 6.135 tone acc: 0.578 initial acc: 0.453 final acc: 0.047
[119,    10] loss: 6.111 tone acc: 0.500 initial acc: 0.516 final acc: 0.141
[120,    10] loss: 6.100 tone acc: 0.516 initial acc: 0.453 final acc: 0.109
[121,    10] loss: 6.070 tone acc: 0.656 initial acc: 0.516 final acc: 0.141

[216,    10] loss: 5.924 tone acc: 0.656 initial acc: 0.547 final acc: 0.125
[217,    10] loss: 5.959 tone acc: 0.531 initial acc: 0.453 final acc: 0.172
[218,    10] loss: 5.949 tone acc: 0.578 initial acc: 0.516 final acc: 0.141
[219,    10] loss: 5.957 tone acc: 0.562 initial acc: 0.516 final acc: 0.109
[220,    10] loss: 5.904 tone acc: 0.625 initial acc: 0.531 final acc: 0.094
[221,    10] loss: 5.936 tone acc: 0.469 initial acc: 0.453 final acc: 0.141
[222,    10] loss: 6.029 tone acc: 0.516 initial acc: 0.562 final acc: 0.094
[223,    10] loss: 5.915 tone acc: 0.594 initial acc: 0.562 final acc: 0.109
[224,    10] loss: 5.869 tone acc: 0.578 initial acc: 0.484 final acc: 0.141
[225,    10] loss: 5.988 tone acc: 0.484 initial acc: 0.484 final acc: 0.109
[226,    10] loss: 5.954 tone acc: 0.609 initial acc: 0.531 final acc: 0.125
[227,    10] loss: 5.898 tone acc: 0.609 initial acc: 0.531 final acc: 0.094
[228,    10] loss: 5.971 tone acc: 0.672 initial acc: 0.500 final acc: 0.188

In [30]:
c_tones = trans.encT.classes_
c_initials = trans.encI.classes_
c_finals = trans.encF.classes_

In [31]:
c_initials

array([' ', 'b', 'g', 'h', 'k', 'kʰ', 'l', 'm', 'n', 'p', 'pʰ', 's', 't',
       'ts', 'tsʰ', 'tʰ', 'z', 'ŋ'], dtype='<U3')

In [32]:
# Test the model

results = []

teochewnet.eval()
with torch.no_grad():
    tone_correct, initial_correct, final_correct = 0, 0, 0
    tone_total, initial_total, final_total = 0, 0, 0
    for _, (x, y_tone, y_initial, y_final) in enumerate(test_generator, 0):
        out1, out2, out3 = teochewnet(Variable(x.float()))
        _, tone_pred = torch.max(out1, 1)
        _, initial_pred = torch.max(out2, 1)
        _, final_pred = torch.max(out3, 1)
        row = [c_tones[tone_pred], c_initials[initial_pred], c_finals[final_pred]]
        row = row + [c_tones[y_tone], c_initials[y_initial], c_finals[y_final]]
        results.append(row)
        tone_total += y_tone.size(0)
        initial_total += y_initial.size(0)
        final_total += y_final.size(0)
        tone_correct += (tone_pred == y_tone).sum().item()
        initial_correct += (initial_pred == y_initial).sum().item()
        final_correct += (final_pred == y_final).sum().item() 
    
    print('tone acc: {}%, inital acc: {}%, final acc: {}%'.format(100*tone_correct/tone_total, \
                                                                 100*initial_correct/initial_total, \
                                                                 100*final_correct/final_total))

tone acc: 58.38509316770186%, inital acc: 59.006211180124225%, final acc: 18.012422360248447%


In [33]:
cols = ['pred_citation', 'pred_initial', 'pred_final', 'citation', 'initial', 'final']
result_df = pd.DataFrame(results, columns=cols)

In [34]:
result_df.sample(30)

,pred_citation,pred_initial,pred_final,citation,initial,final
37,33,tʰ,o,21,tʰ,oʔ
97,213,p,ou,213,pʰ,ue
132,53,,ĩã∼,53,,uŋ
140,33,s,i,21,s,ueʔ
59,35,,iu,4,g,ueʔ
141,55,l,ou,213,l,iu
96,33,t,u,4,t,ieʔ
113,33,tʰ,o,21,tʰ,ik
149,213,k,ɯŋ,213,kʰ,ɯŋ
39,213,p,i,21,p,iaʔ


In [35]:
result_df[result_df['final']==result_df['pred_final']]

,pred_citation,pred_initial,pred_final,citation,initial,final
0,33,k,ou,33,kʰ,ou
9,55,l,aŋ,55,l,aŋ
13,213,p,aŋ,213,p,aŋ
26,55,p,ue,55,p,ue
32,213,kʰ,ou,213,kʰ,ou
36,55,l,o,55,l,o
43,55,l,aŋ,33,l,aŋ
45,53,t,iəu,55,h,iəu
61,55,k,iu,55,k,iu
62,55,l,aŋ,55,l,aŋ


In [36]:
trans.teochew_df['citation_teo'] = trans.teochew_df['citation_teo'].astype(str)
result_df[cols] = result_df[cols].astype(str)

In [37]:
merged_pd = pd.merge(trans.teochew_df, result_df, how='left', left_on=['citation_teo','initial_teo','final_teo'], \
         right_on = ['citation', 'initial', 'final'])
merged_pd[(merged_pd['pred_citation'].notnull())&(merged_pd['pred_final'].notnull())].sample(30)

,BENZI_IN_SOURCE_teo,citation_teo,initial_teo,final_teo,pred_citation,pred_initial,pred_final,citation,initial,final
397,帽,11,b,o,4,m,e,11,b,o
325,鱗,55,l,aŋ,55,l,aŋ,55,l,aŋ
517,月,4,g,ueʔ,35,,iu,4,g,ueʔ
542,簿,35,pʰ,ou,213,p,ou,35,pʰ,ou
802,渣,33,ts,a,33,t,ou,33,ts,a
317,電,35,t,ieŋ,213,t,ɯŋ,35,t,ieŋ
210,行,55,h,aŋ,55,,iŋ,55,h,aŋ
252,聾,55,l,aŋ,55,l,aŋ,55,l,aŋ
299,生,33,s,ẽ∼,33,s,uŋ,33,s,ẽ∼
305,爸,33,p,a,4,p,ak,33,p,a
